In [ ]:
#Python Runtime
from google.colab import drive
drive.mount('/content/drive')

#system("tar xf /content/drive/MyDrive/Colab Notebooks/lib/library.tar.gz")
#.libPaths('/content/usr/local/lib/R/site-library/')

In [ ]:
#R Runtime
install.packages("ggplot2")
install.packages("dplyr")
#system("apt-get install gfortran")
#install.packages("ggpubr")
install.packages("tidyr")
devtools::install_github("caleblareau/BuenColors")
install.packages('patchwork')
install.packages('cowplot')
devtools::install_github("jakelawlor/PNWColors")

library(PNWColors)
library(tidyr)
library(ggplot2)
library(dplyr)
#library(ggpubr)
library(BuenColors)
library(cowplot)
library(patchwork)

In [ ]:
#tar("library.tar.gz", "/usr/local/lib/R/site-library")

In [ ]:
data_path="/content/drive/MyDrive/Manuscripts/2023_UKBB-BBJ-GTEx_MPRA/2023_UKBB_GTEx_MPRA_Manuscript/"
output_path="/content/drive/MyDrive/Temp/"

gtex_mpra_paired_final20230117 <- read.delim(paste0(data_path,"Data/MPRA/gtex_mpra_final20230117.txt.gz"),header=TRUE,stringsAsFactors = FALSE,check.names=FALSE)
traits_mpra_paired_final20230117 <- read.delim(paste0(data_path,"Data/MPRA/traits_mpra_final20230117.txt.gz"),header=TRUE,stringsAsFactors = FALSE,check.names=FALSE)
haplotypes_data <- read.table(paste0(data_path,"Data/MPRA/haplos_merged_20240202.txt"), sep = '\t', header = TRUE)

In [ ]:
if (!dir.exists(paste0(output_path,"plots/"))) {
  dir.create(paste0(output_path,"plots/"))
} else {
  cat("Directory already exists!\n")
}

if (!dir.exists(paste0(output_path,"haplo_plots/"))) {
  dir.create(paste0(output_path,"haplo_plots/"))
} else {
  cat("Directory already exists!\n")
}

In [ ]:
desired_ids <- c("chr12:31226835:T:A", "chr13:50432800:T:C", "chr14:65175364:A:C", "chr15:55526886:C:T", "chr16:193224:G:A", "chr19:58662235:C:T", "chr19:58662254:T:C", "chr2:191049403:G:A", "chr20:45944659:G:C", "chr20:5485337:G:A", "chr20:61464184:C:T", "chr21:37572401:T:G", "chr22:19132380:A:T", "chr22:19132385:G:A", "chr22:50742721:T:A", "chr3:42306446:C:T", "chr5:95163449:G:A", "chr6:7143859:C:A", "chr7:105416560:C:A", "chr7:139875230:C:T")

subset_gtex <- gtex_mpra_paired_final20230117 %>% filter(variant %in% desired_ids)

glimpse(subset_gtex)


In [ ]:
selected_columns <- c("variant","cell_type","A_log2FC", "B_log2FC", "A_log2FC_SE", "B_log2FC_SE")  # Replace with actual column names

pre_filtered_data <- subset_gtex %>%
  dplyr::select(all_of(selected_columns))

#plot_data_melted <- pre_filtered_data %>%
#  pivot_longer(
#    cols = c("A_log2FC", "B_log2FC"),
#    names_to = "log2FC_type",
#    values_to = "log2FC"
#  )

  melted_data <- pre_filtered_data %>%
  pivot_longer(
    cols = c(A_log2FC, A_log2FC_SE, B_log2FC, B_log2FC_SE),
    names_to = c("set", ".value"),
    names_pattern = "(A|B)_(.*)"
  ) %>%
  distinct()

#melted_data

In [ ]:
# Setup theme
my_theme <-
  BuenColors::pretty_plot(fontsize = 8) +
  BuenColors::L_border() +
  theme(
    plot.background = element_blank(),
    plot.margin = margin(0, 0.1, 0, 0.1, unit = "cm"),
    plot.title = element_text(hjust = 4e-3, margin = margin(b = -12)),
    # legend.position = "none",
    legend.position = c(0.9, 1),
    legend.justification = c(1, 1),
    legend.title = element_text(margin = margin(0, 0, 0, 0)),
    legend.background = element_blank(),
    legend.key.size = unit(0.2, "cm")

  )

In [ ]:



font_family <- "Arial"
title_size <- 16
axis_title_size <- 16
axis_text_size <- 12
legend_title_size <- 16
legend_text_size <- 12
legend_key_size <- 1

clrs = c(
  pnw_palette('Bay',8,type='continuous')[5],
  pnw_palette('Bay',8,type='continuous')[3],
  pnw_palette('Starfish')[5],
  pnw_palette('Bay',8,type='continuous')[1],
  pnw_palette('Bay',8,type='continuous')[8],
  'black'
)

names(clrs) = c("A549", "HCT116", "HEPG2", "K562","SKNSH","any")
base_colors<-clrs

lighten_color <- function(color, amount = 0.3) {
  alpha(color, amount)
}

color_palette <- c()
for (cell_type in names(base_colors)) {
  color_palette[paste0(cell_type, "_A")] <- base_colors[cell_type]
  color_palette[paste0(cell_type, "_B")] <- lighten_color(base_colors[cell_type])
}

uniq_ID=unique(melted_data$variant)

for (i in 1:length(uniq_ID)) {

cur_ID=uniq_ID[i]
print(i)
print(cur_ID)

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

#print(filtered_data)

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))


p1<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2", "K562"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")
filtered_data$label <- factor(filtered_data$label, levels = c("Ref", "Alt"))

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p2<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")
filtered_data$label <- factor(filtered_data$label, levels = c("Ref", "Alt"))

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p3<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("K562"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")
filtered_data$label <- factor(filtered_data$label, levels = c("Ref", "Alt"))

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p4<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

plt_combined <- p1 +  p2 + p3 / p4 + p3 + p4 + p2 + p3 + p4 + plot_annotation(title = paste0(cur_ID,"- GTEx")) + plot_layout(ncol=8,nrow=1, widths=c(2,1,1,1,1,2,0.5,0.5),heights=c(2,2,2,2,2,2,2,2))
p1
p2
plt_combined
print(cur_ID)
cowplot::save_plot(
  paste0(output_path,"plots/",cur_ID,"_activity.gtex.pdf"),
  plt_combined,
  base_height = 6,
  base_width = 25,
  device = cairo_pdf
)
}

In [ ]:
##RUN WITH UKBB/BBJ

desired_ids <- c("chr12:31226835:T:A", "chr13:50432800:T:C", "chr14:65175364:A:C", "chr15:55526886:C:T", "chr16:193224:G:A", "chr19:58662235:C:T", "chr19:58662254:T:C", "chr2:191049403:G:A", "chr20:45944659:G:C", "chr20:5485337:G:A", "chr20:61464184:C:T", "chr21:37572401:T:G", "chr22:19132380:A:T", "chr22:19132385:G:A", "chr22:50742721:T:A", "chr3:42306446:C:T", "chr5:95163449:G:A", "chr6:7143859:C:A", "chr7:105416560:C:A", "chr7:139875230:C:T")

subset_gtex <- traits_mpra_paired_final20230117 %>% filter(variant %in% desired_ids)

selected_columns <- c("variant","cell_type","A_log2FC", "B_log2FC", "A_log2FC_SE", "B_log2FC_SE")  # Replace with actual column names

pre_filtered_data <- subset_gtex %>%
  dplyr::select(all_of(selected_columns))

#plot_data_melted <- pre_filtered_data %>%
#  pivot_longer(
#    cols = c("A_log2FC", "B_log2FC"),
#    names_to = "log2FC_type",
#    values_to = "log2FC"
#  )

  melted_data <- pre_filtered_data %>%
  pivot_longer(
    cols = c(A_log2FC, A_log2FC_SE, B_log2FC, B_log2FC_SE),
    names_to = c("set", ".value"),
    names_pattern = "(A|B)_(.*)"
  ) %>%
  distinct()

#melted_data

font_family <- "Arial"
title_size <- 16
axis_title_size <- 16
axis_text_size <- 12
legend_title_size <- 16
legend_text_size <- 12
legend_key_size <- 1

clrs = c(
  pnw_palette('Bay',8,type='continuous')[5],
  pnw_palette('Bay',8,type='continuous')[3],
  pnw_palette('Starfish')[5],
  pnw_palette('Bay',8,type='continuous')[1],
  pnw_palette('Bay',8,type='continuous')[8],
  'black'
)

names(clrs) = c("A549", "HCT116", "HEPG2", "K562","SKNSH","any")
base_colors<-clrs

lighten_color <- function(color, amount = 0.3) {
  alpha(color, amount)
}

color_palette <- c()
for (cell_type in names(base_colors)) {
  color_palette[paste0(cell_type, "_A")] <- base_colors[cell_type]
  color_palette[paste0(cell_type, "_B")] <- lighten_color(base_colors[cell_type])
}

uniq_ID=unique(melted_data$variant)

for (i in 1:length(uniq_ID)) {


cur_ID=uniq_ID[i]
print(i)
print(cur_ID)

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

#print(filtered_data)

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))


p1<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2", "K562"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p2<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p3<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("K562"))

filtered_data$label <- ifelse(filtered_data$set == "A", "Ref", "Alt")

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$set, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p4<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

plt_combined <- p1 +  p2 + p3 / p4 + p3 + p4 + p2 + p3 + p4 + plot_annotation(title = paste0(cur_ID,"- GTEx")) + plot_layout(ncol=8,nrow=1, widths=c(2,1,1,1,1,2,0.5,0.5),heights=c(2,2,2,2,2,2,2,2))
p1
p2
plt_combined
print(cur_ID)
cowplot::save_plot(
  paste0(output_path,"plots/",cur_ID,"_activity.ukbb.pdf"),
  plt_combined,
  base_height = 6,
  base_width = 25,
  device = cairo_pdf
)
}

In [ ]:
head(melted_data)

In [ ]:
head(filtered_data)

In [ ]:
head(haplotypes_data)

In [ ]:
melted_data <- haplotypes_data
melted_data <- melted_data %>% rename(variant = sat_ref_parent)
melted_data <- melted_data %>% rename(log2FC = log2FC_window)
melted_data <- melted_data %>% rename(log2FC_SE = log2FC_window_SE)

#melted_data

font_family <- "Arial"
title_size <- 16
axis_title_size <- 16
axis_text_size <- 12
legend_title_size <- 16
legend_text_size <- 12
legend_key_size <- 1

clrs = c(
  pnw_palette('Bay',8,type='continuous')[5],
  pnw_palette('Bay',8,type='continuous')[3],
  pnw_palette('Starfish')[5],
  pnw_palette('Bay',8,type='continuous')[1],
  pnw_palette('Bay',8,type='continuous')[8],
  'black'
)

names(clrs) = c("A549", "HCT116", "HEPG2", "K562","SKNSH","any")
base_colors<-clrs

lighten_color_AB <- function(color, amount = 0.70) {
  alpha(color, amount)
}

lighten_color_BA <- function(color, amount = 0.40) {
  alpha(color, amount)
}

lighten_color_BB <- function(color, amount = 0.10) {
  alpha(color, amount)
}

color_palette <- c()
for (cell_type in names(base_colors)) {
  color_palette[paste0(cell_type, "_II")] <- base_colors[cell_type]
  color_palette[paste0(cell_type, "_ID")] <- lighten_color_AB(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DI")] <- lighten_color_BA(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DD")] <- lighten_color_BB(base_colors[cell_type])
}

uniq_ID=unique(melted_data$variant)

#i=1

for (i in 1:length(uniq_ID)) {
cur_ID=uniq_ID[i]
print(i)
print(cur_ID)

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))
filtered_data$label<-filtered_data$allele

p1<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

#K562 & HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2", "K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p2<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")


### HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))


p3<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

##K562 Cells Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))



p4<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

plt_combined <- p1 +  p2 + p3 / p4 + p3 + p4 + p2 + p3 + p4 + plot_annotation(title = paste0(cur_ID,"")) + plot_layout(ncol=8,nrow=1, widths=c(2,1,1,1,1,2,0.5,0.5),heights=c(2,2,2,2,2,2,2,2))
p1
p2
plt_combined
print(cur_ID)
cowplot::save_plot(
  paste0(output_path,"haplo_plots/",cur_ID,"_haplotypes_orig.pdf"),
  plt_combined,
  base_height = 6,
  base_width = 25,
  device = cairo_pdf
)
}


In [ ]:
head(filtered_data)

In [ ]:
color_palette

In [ ]:
p4

In [ ]:
melted_data <- haplotypes_data
melted_data <- melted_data %>% rename(variant = sat_ref_parent)
melted_data <- melted_data %>% rename(log2FC = log2FC_meta)
melted_data <- melted_data %>% rename(log2FC_SE = log2FC_meta_SE)

#melted_data

font_family <- "Arial"
title_size <- 16
axis_title_size <- 16
axis_text_size <- 12
legend_title_size <- 16
legend_text_size <- 12
legend_key_size <- 1

clrs = c(
  pnw_palette('Bay',8,type='continuous')[5],
  pnw_palette('Bay',8,type='continuous')[3],
  pnw_palette('Starfish')[5],
  pnw_palette('Bay',8,type='continuous')[1],
  pnw_palette('Bay',8,type='continuous')[8],
  'black'
)

names(clrs) = c("A549", "HCT116", "HEPG2", "K562","SKNSH","any")
base_colors<-clrs

lighten_color_AB <- function(color, amount = 0.70) {
  alpha(color, amount)
}

lighten_color_BA <- function(color, amount = 0.40) {
  alpha(color, amount)
}

lighten_color_BB <- function(color, amount = 0.10) {
  alpha(color, amount)
}

color_palette <- c()
for (cell_type in names(base_colors)) {
  color_palette[paste0(cell_type, "_II")] <- base_colors[cell_type]
  color_palette[paste0(cell_type, "_ID")] <- lighten_color_AB(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DI")] <- lighten_color_BA(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DD")] <- lighten_color_BB(base_colors[cell_type])
}

uniq_ID=unique(melted_data$variant)

#i=1

for (i in 1:length(uniq_ID)) {
cur_ID=uniq_ID[i]
print(i)
print(cur_ID)

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))
filtered_data$label<-filtered_data$allele

p1<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

#K562 & HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2", "K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p2<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")


### HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))


p3<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

##K562 Cells Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))



p4<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

plt_combined <- p1 +  p2 + p3 / p4 + p3 + p4 + p2 + p3 + p4 + plot_annotation(title = paste0(cur_ID,"")) + plot_layout(ncol=8,nrow=1, widths=c(2,1,1,1,1,2,0.5,0.5),heights=c(2,2,2,2,2,2,2,2))
p1
p2
plt_combined
print(cur_ID)
cowplot::save_plot(
  paste0(output_path,"haplo_plots/",cur_ID,"_haplotypes_meta.pdf"),
  plt_combined,
  base_height = 6,
  base_width = 25,
  device = cairo_pdf
)
}

In [ ]:
melted_data <- haplotypes_data
melted_data <- melted_data %>% rename(variant = sat_ref_parent)
melted_data <- melted_data %>% rename(log2FC = log2FC_satmut)
melted_data <- melted_data %>% rename(log2FC_SE = log2FC_satmut_SE)

#melted_data

font_family <- "Arial"
title_size <- 16
axis_title_size <- 16
axis_text_size <- 12
legend_title_size <- 16
legend_text_size <- 12
legend_key_size <- 1

clrs = c(
  pnw_palette('Bay',8,type='continuous')[5],
  pnw_palette('Bay',8,type='continuous')[3],
  pnw_palette('Starfish')[5],
  pnw_palette('Bay',8,type='continuous')[1],
  pnw_palette('Bay',8,type='continuous')[8],
  'black'
)

names(clrs) = c("A549", "HCT116", "HEPG2", "K562","SKNSH","any")
base_colors<-clrs

lighten_color_AB <- function(color, amount = 0.70) {
  alpha(color, amount)
}

lighten_color_BA <- function(color, amount = 0.40) {
  alpha(color, amount)
}

lighten_color_BB <- function(color, amount = 0.10) {
  alpha(color, amount)
}

color_palette <- c()
for (cell_type in names(base_colors)) {
  color_palette[paste0(cell_type, "_II")] <- base_colors[cell_type]
  color_palette[paste0(cell_type, "_ID")] <- lighten_color_AB(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DI")] <- lighten_color_BA(base_colors[cell_type])
  color_palette[paste0(cell_type, "_DD")] <- lighten_color_BB(base_colors[cell_type])
}

uniq_ID=unique(melted_data$variant)

#i=1

for (i in 1:length(uniq_ID)) {
cur_ID=uniq_ID[i]
print(i)
print(cur_ID)

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))
filtered_data$label<-filtered_data$allele

p1<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

#K562 & HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2", "K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))

p2<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")


### HepG2 Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("HEPG2"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))


p3<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

##K562 Cells Only

filtered_data <- melted_data %>%
  filter(variant == cur_ID)

filtered_data <- filtered_data %>%
  filter(cell_type %in% c("K562"))

filtered_data$label<-filtered_data$allele

filtered_data$cell_type_set <- interaction(filtered_data$cell_type, filtered_data$allele_recode, sep = "_")
filtered_data$cell_type_set <- factor(filtered_data$cell_type_set, levels = levels(filtered_data$cell_type_set))



p4<-ggplot(filtered_data, aes(x = cell_type, y = log2FC, fill = cell_type_set)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8), width = 0.7) +
  geom_errorbar(aes(ymin = log2FC - log2FC_SE, ymax = log2FC + log2FC_SE),
                position = position_dodge(width = 0.8), width = 0) +
  geom_text(aes(y=-0, label = label), size=5, position = position_dodge(width = 0.8),
            angle = 0, vjust = 0) +
  labs(y = "Activity - Log2(RNA/DNA)", x = "Cell Type") +
  pretty_plot() +
  my_theme +
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +  # Adjust x-axis limits and expansion
  scale_x_discrete(expand = expansion(add = c(0.5, 0.5))) + # Reduce white space above and below bars
  theme(
          text = element_text(family = font_family),  # Set font family for the entire plot
          plot.title = element_text(size = title_size),
          axis.title = element_text(size = axis_title_size),
          axis.text = element_text(size = axis_text_size),
          axis.text.x = element_text(angle = 0, hjust=0.5),
          axis.title.x = element_text(angle=0, hjust=0.5)
    ) +
  scale_fill_manual(values = color_palette) +
  theme(legend.position = "none")

plt_combined <- p1 +  p2 + p3 / p4 + p3 + p4 + p2 + p3 + p4 + plot_annotation(title = paste0(cur_ID,"")) + plot_layout(ncol=8,nrow=1, widths=c(2,1,1,1,1,2,0.5,0.5),heights=c(2,2,2,2,2,2,2,2))
p1
p2
plt_combined
print(cur_ID)
cowplot::save_plot(
  paste0(output_path,"haplo_plots/",cur_ID,"_haplotypes_SatMut.pdf"),
  plt_combined,
  base_height = 6,
  base_width = 25,
  device = cairo_pdf
)
}